# TODOs

1. Check if all there is 1-1 mapping for users/links and posts/links

# Import needed libraries

In [1]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
from lxml import html
import random
import json, time, re, os
import itertools
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import re
from dateutil.parser import parse
import time
from datetime import datetime, timedelta

from contextlib import contextmanager

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import wget

%matplotlib inline

# Params

In [2]:
# starts chromedriver to make the usage of Chrome possible
# 1.Check the version of chrome browser.
# 2.download chromedriver of same version from https://sites.google.com/chromium.org/driver/home
# 3.Move near to the notebook and run the code.

# NB!!! You should be authorized on the website you parse in the browser you 
# use. In my case I have authorized on forum.littleone.ru in Chrome browser

files = os.listdir()
file = [file for file in files if file == 'chromedriver.exe'][0]
os.startfile(file)

In [3]:
top_n_pages = -1
forum = 'https://forum.littleone.ru/'
# data_path = 'E:\Files\Data\littleone'
# desktop
raw_data_path = 'D:\Data\littleone_html'
data_path_csv = 'D:\Data\littleone_csv_with_names'

# Shared funcs

In [4]:
def driver2soup(driver):
    return BeautifulSoup(driver.page_source)


def read_html_with_webdriver(url_to_read, wait_sec=7, headless=True, 
                             disable_extensions=True, close_page=False):
    
    if headless or disable_extensions:
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        if disable_extensions:
            chrome_options.add_argument("--disable-extensions")
    #chrome_options.add_argument("--disable-gpu")
        driver = webdriver.Chrome(options=chrome_options)
    else:
        driver = webdriver.Chrome()
    
    driver.get(url_to_read)
    time.sleep(wait_sec)

    soup = driver2soup(driver)
    
    if close_page:
        driver.close()
        return soup
    
    return soup, driver

In [5]:
def get_posts_from_page(soup):
    posts_ol = soup.find('ol', {'id': 'posts'})
    assert posts_ol is not None, 'Posts not found on page. Soup:\n{}'.format(soup)
    return posts_ol.find_all('li', {'class': 'postbitlegacy postbitim postcontainer old'})

def get_post_text(post):
    return post.find('div', 'postdetails').find('div', 'content').find('blockquote', 'postcontent restore').text

def get_post_texts_from_page(soup):
    posts = get_posts_from_page(soup)
    return [get_post_text(post) for post in posts]

In [6]:
# to ensure that page is loaded and we may read it
# http://www.obeythetestinggoat.com/how-to-get-selenium-to-wait-for-page-load-after-a-click.html

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import \
    staleness_of


@contextmanager
def wait_for_page_load(driver, timeout=30):
    old_page = driver.find_element_by_tag_name('html')
    yield
    WebDriverWait(driver, timeout,
                  poll_frequency=0.1).until(staleness_of(old_page))


def move_to_next_page(driver):
    #print('moving from page', driver.find_element_by_tag_name('html').id)
    try:
        with wait_for_page_load(driver):
            driver.find_element_by_xpath('//*[@rel="next"]').click()
        #print('moved to', driver.find_element_by_tag_name('html').id)
    except:
        print('Unable to get next page')

In [7]:
def flatten_list(ls):
    return list(itertools.chain.from_iterable(ls))

In [8]:
def get_user_extra_info(post):
    extra_info = post.find('dl', {'class': 'userinfo_extra'})
    #print('extra_info', extra_info)
    if extra_info is None:
        return {'guest': True}
    else:
        info_dict = dict(
            zip(
                list(map(lambda x: x.text,
                         extra_info.find_all("dt"))),  # Extra field names
                list(map(lambda x: x.text.replace('\xa0', ''),
                     extra_info.find_all("dd")))  # Extra field names
            ))
        info_dict.update({'guest': False})
        return info_dict


def get_user_info_from_post(post):
    user_info = dict()
    user_info['user_title'] = post.find('span', {'class': 'usertitle'}).text
    try:
        user_info['user_name'] = post.find('div', 
                                           {'class': 'username_container'}).find("strong").text
        user_info['user_link'] = forum + post.find('div', {'class': 'username_container'}).find("a").attrs['href']
    except:
        user_info['user_name'] = post.find('span', {
            'class': 'username guest'
        }).text
        #print('!'*100, post)
    user_info.update(get_user_extra_info(post))
    return user_info


def parse_date(date_string, file_path=None):
    try:
        return parse(date_string, dayfirst=True)
    except:
#         print('failed to parse date using dateutil.parser') # debug
        try:
            if file_path is not None:
                file_created_date = time.ctime(os.path.getctime(file_path))
                file_created_date = parse(file_created_date).date()
            else:
                file_created_date = parse(time.ctime()).date()
            post_time = datetime.strptime(
                date_string.split('\xa0')[-1], '%H:%M').time()
            post_datetime = datetime.combine(file_created_date, post_time)
            if re.findall('вчера', date_string):
                post_datetime -= timedelta(days=1)
            elif re.findall('сегодня', date_string):
                pass
            else:
                raise ValueError('Unexpected characters in date_string:',
                                 date_string)


#             print('file was created', file_created_date,
#                   '\ndate_string', date_string,
#                   '\npost_time', post_time,
#                   '\npost_datetime', post_datetime, f"D{post_datetime.day},M{post_datetime.month},Y{post_datetime.year}")
            return post_datetime
        except:
            raise ValueError('Unreadable format of date:', date_string)


def get_info_from_post_header(post, file_path=None):
    header_info = dict()
    header_info['post_datetime'] = parse_date(post.find(
        'span', {'class': 'date'}).text, file_path=file_path)

    div = post.find('div', {'class': 'posthead'}).find("div")
    #print(div, '\n')
    if div is not None:
        try:
            header_info['answer_to_user'] = re.findall(
                'ответ для ([\S\s]*) , на сообщение « ', div.text)[0]

            header_info['answer_to_post_head'] = re.findall(
                ' , на сообщение « ([\S\s]*) »', div.text)[0]

            links = div.find_all('a', href=True)
            if len(links) == 2:
                header_info['answer_to_user_link'], header_info['answer_to_post_link'] \
                    = [forum + link.attrs['href'] for link in links]
            else:
                header_info[
                    'answer_to_post_link'] = forum + links[0].attrs['href']
        except:
            print('!' * 20, '\n', div, '\n', file_path, '\n',
                  '!' * 20)  # debug

    return header_info


def get_all_info_from_page(page_soup, file_path=None):
    posts = get_posts_from_page(page_soup)

    page_texts = [get_post_text(post) for post in posts]
    texts_df = pd.DataFrame({'post_text': page_texts})

    users_info = [get_user_info_from_post(post) for post in posts]
    users_info_df = pd.DataFrame(users_info)

    header_info = [
        get_info_from_post_header(post, file_path=file_path) for post in posts
    ]
    header_info_df = pd.DataFrame(header_info)

    return pd.concat([texts_df, users_info_df, header_info_df],
                     axis=1)  #.astype({'Сообщений': int}) NaNs occurs

In [9]:
# Add functionality for continuing stopped thread

def get_all_info_from_file(file_path):
    with open(file_path, 'r') as f:
        contents = f.read()
        soup = BeautifulSoup(contents)
        all_info_from_page = get_all_info_from_page(soup, file_path)
        return all_info_from_page

## Testing

### Check parsing from file

In [10]:
test_path = os.path.join(raw_data_path, str(3275945), str(133))
test_path

'D:\\Data\\littleone_html\\3275945\\133'

In [11]:
df = get_all_info_from_file(test_path)
df

,post_text,user_title,user_name,user_link,Регистрация,Адрес,Сообщений,guest,post_datetime,answer_to_user,answer_to_post_head,answer_to_user_link,answer_to_post_link
0,"\n\t\t\t\t\t\t\tНа сколько знаю я, в Канаде не...",\n\t\t\t\tХранитель\n\t\t\t,tatash,https://forum.littleone.ru/member.php?u=83506&...,14.06.2008,5*,12043,False,2014-11-07 13:18:00,ТАТТО,Прочтите здесь...,https://forum.littleone.ru/member.php?userid=3...,https://forum.littleone.ru/showthread.php?p=11...
1,\n\n\n\n\n\n Сообщение от tatash\n\n\nНа сколь...,\n\t\t\t\tХранитель\n\t\t\t,ТАТТО,https://forum.littleone.ru/member.php?u=31781&...,03.03.2007,м. Приморская,14676,False,2014-11-07 13:41:00,tatash,"На сколько знаю я, в Канаде нет...",https://forum.littleone.ru/member.php?userid=8...,https://forum.littleone.ru/showthread.php?p=11...
2,\n\n\n\n\n\n Сообщение от канри\n\n\nне знаю к...,\n\t\t\t\tЭлита\n\t\t\t,agusya,https://forum.littleone.ru/member.php?u=44923&...,30.08.2007,Остров,2671,False,2014-11-07 14:06:00,канри,не знаю как поддержать даже.. сама...,https://forum.littleone.ru/member.php?userid=6...,https://forum.littleone.ru/showthread.php?p=11...
3,\n\n\n\n\n\n Сообщение от ТАТТО\n\n\nА разве я...,\n\t\t\t\tХранитель\n\t\t\t,tatash,https://forum.littleone.ru/member.php?u=83506&...,14.06.2008,5*,12043,False,2014-11-07 16:26:00,ТАТТО,А разве я про это дала ссылку?,https://forum.littleone.ru/member.php?userid=3...,https://forum.littleone.ru/showthread.php?p=11...
4,\n\n\n\n\n\n Сообщение от Юбиляр\n\n\nВ смысле...,\n\t\t\t\tBanned\n\t\t\t,канри,https://forum.littleone.ru/member.php?u=632441...,29.05.2013,NaN,9190,False,2014-11-08 02:48:00,Юбиляр,В смысле нереально ? :001: учится в...,https://forum.littleone.ru/member.php?userid=7...,https://forum.littleone.ru/showthread.php?p=11...
5,\n\n\n\n\n\n Сообщение от agusya\n\n\nА общага...,\n\t\t\t\tBanned\n\t\t\t,канри,https://forum.littleone.ru/member.php?u=632441...,29.05.2013,NaN,9190,False,2014-11-08 02:55:00,agusya,А общага тоже не бесплатна. И в...,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=11...
6,\n\n\n\n\n\n Сообщение от tatash\n\n\nС учета ...,\n\t\t\t\tBanned\n\t\t\t,канри,https://forum.littleone.ru/member.php?u=632441...,29.05.2013,NaN,9190,False,2014-11-08 04:37:00,tatash,С учета какого? \nРегистрационного? Или...,https://forum.littleone.ru/member.php?userid=8...,https://forum.littleone.ru/showthread.php?p=11...
7,"\n\t\t\t\t\t\t\tЯ слышала, что в Финляндии отм...",\n\t\t\t\tГость\n\t\t\t,SashaLeon,https://forum.littleone.ru/member.php?u=711654...,24.10.2014,NaN,18,False,2014-11-09 13:41:00,канри,нет ...не на Винского.. Винский в...,https://forum.littleone.ru/member.php?userid=6...,https://forum.littleone.ru/showthread.php?p=11...
8,\n\n\n\n\n\n Сообщение от канри\n\n\nМожно спр...,\n\t\t\t\tСтарожил\n\t\t\t,Королева ОГНЯ,https://forum.littleone.ru/member.php?u=57946&...,21.12.2007,Питерские просторы,1456,False,2014-11-09 13:59:00,канри,"Можно спросить, ну просто ради...",https://forum.littleone.ru/member.php?userid=6...,https://forum.littleone.ru/showthread.php?p=11...
9,\n\n\n\n\n\t\t\t\n\t\t\t\tЭти направления были...,\n\t\t\t\tBanned\n\t\t\t,канри,https://forum.littleone.ru/member.php?u=632441...,29.05.2013,NaN,9190,False,2014-11-10 01:54:00,Katusha88,Высшее образование заграницей БЕСПЛАТНО...,https://forum.littleone.ru/member.php?userid=9...,https://forum.littleone.ru/showthread.php?p=49...


In [12]:
df['post_datetime'].dt.day

0     7
1     7
2     7
3     7
4     8
5     8
6     8
7     9
8     9
9    10
Name: post_datetime, dtype: int64

### Check parsing with driver

In [13]:
thread_url = 'https://forum.littleone.ru/showthread.php?t=3275945&page=942'
thread_soup = read_html_with_webdriver(thread_url, close_page=True)

In [14]:
get_all_info_from_page(thread_soup).T

,0,1,2,3,4,5,6,7,8,9
post_text,"\n\t\t\t\t\t\t\tПодскажите пожалуйста, если не...",\n\t\t\t\t\t\t\tСегодня вывезла своего студент...,\n\n\n\n\n\n Сообщение от Fairy\n\n\nПодскажит...,\n\n\n\n\n\n Сообщение от Живая бабочка\n\n\nо...,\n\n\n\n\n\n Сообщение от iceman\n\n\nУ сабжек...,\n\n\n\n\n\n Сообщение от Живая бабочка\n\n\nС...,\n\t\t\t\t\t\t\tбольшинство экз.центров в сша ...,"\n\t\t\t\t\t\t\tНичего себе, 1240 считается вы...",\n\n\n\n\n\n Сообщение от iceman\n\n\nНичего с...,\n\n\n\n\n\n Сообщение от sirop\n\n\nбольшинст...
user_title,\n\t\t\t\tМега-элита\n\t\t\t,\n\t\t\t\tМега-элита\n\t\t\t,\n\t\t\t\tСтарожил\n\t\t\t,\n\t\t\t\tНаш человек\n\t\t\t,\n\t\t\t\tСтарожил\n\t\t\t,\n\t\t\t\tНаш человек\n\t\t\t,\n\t\t\t\tМега-элита\n\t\t\t,\n\t\t\t\tНаш человек\n\t\t\t,\n\t\t\t\tСтарожил\n\t\t\t,\n\t\t\t\tСтарожил\n\t\t\t
user_name,Fairy,Fairy,Живая бабочка,iceman,Живая бабочка,iceman,sirop,iceman,Живая бабочка,Живая бабочка
user_link,https://forum.littleone.ru/member.php?u=28653&...,https://forum.littleone.ru/member.php?u=28653&...,https://forum.littleone.ru/member.php?u=250853...,https://forum.littleone.ru/member.php?u=212676...,https://forum.littleone.ru/member.php?u=250853...,https://forum.littleone.ru/member.php?u=212676...,https://forum.littleone.ru/member.php?u=297364...,https://forum.littleone.ru/member.php?u=212676...,https://forum.littleone.ru/member.php?u=250853...,https://forum.littleone.ru/member.php?u=250853...
Регистрация,22.01.2007,22.01.2007,09.01.2010,06.10.2009,09.01.2010,06.10.2009,25.05.2010,06.10.2009,09.01.2010,09.01.2010
Сообщений,4803,4803,1292,6375,1292,6375,3205,6375,1292,1292
guest,False,False,False,False,False,False,False,False,False,False
post_datetime,2020-09-12 00:53:00,2020-09-12 00:58:00,2020-09-12 13:25:00,2020-09-12 13:56:00,2020-09-12 14:28:00,2020-09-12 15:00:00,2020-09-12 17:23:00,2020-09-12 18:09:00,2020-09-12 18:13:00,2020-09-12 18:19:00
answer_to_user,Живая бабочка,пони777,Fairy,Живая бабочка,iceman,Живая бабочка,Katusha88,Katusha88,iceman,sirop
answer_to_post_head,"Мы в США, но по американским законам,...",А Вас тоже через границу пропустили с...,"Подскажите пожалуйста, если не секрет,...","очень высокий. Общий sat был, если не...",У сабжект сат максимальный возмлжный...,Сабджект тесты Же по 2 предметам надо...,Высшее образование заграницей БЕСПЛАТНО...,Высшее образование заграницей БЕСПЛАТНО...,"Ничего себе, 1240 считается выше...",большинство экз.центров в сша открыто....


In [25]:
posts = get_posts_from_page(thread_soup)

In [26]:
div = posts[6].find('div', {'class' : 'posthead'}).find("div")
print(div.text)
re.findall('на сообщение « ([\S\s]*) »', div.text)

ответ для Katusha88 , на сообщение « Высшее образование заграницей БЕСПЛАТНО... » 


['Высшее образование заграницей БЕСПЛАТНО...']

In [27]:
get_user_info_from_post(posts[5])

{'user_title': '\n\t\t\t\tНаш человек\n\t\t\t',
 'user_name': 'iceman',
 'user_link': 'https://forum.littleone.ru/member.php?u=212676&s=a6819aaeba40c01222eb95532eaaf2e0',
 'Регистрация': '06.10.2009',
 'Сообщений': '6375',
 'guest': False}

In [28]:
get_info_from_post_header(posts[5], None)

{'post_datetime': datetime.datetime(2020, 9, 12, 15, 0),
 'answer_to_user': 'Живая бабочка',
 'answer_to_post_head': 'Сабджект тесты  Же по 2 предметам надо...',
 'answer_to_user_link': 'https://forum.littleone.ru/member.php?userid=250853',
 'answer_to_post_link': 'https://forum.littleone.ru/showthread.php?p=172297815#post172297815'}

In [29]:
parse('09.11.2014')

datetime.datetime(2014, 9, 11, 0, 0)

# Parse directly to csv. Not fast, not used

In [45]:
def get_save_freq(n_pages):
    if n_pages < 1000: return 500
    elif n_pages < 10000: return 5000
    else: return 10000


def get_pages_num(thread_soup):
    # it seems that from the moment of first development the engine changed a bit
#     last_page_tag = thread_soup.find(
#         "a", title=lambda title: title and title.startswith("Последняя"))
#     top_n_pages = int(re.findall('из ([\d\s]*)',
#                                  last_page_tag.attrs['title'])[0].replace(u'\xa0', ''))
    # current version
    try:
        pages_regex = 'Страница [\d]* из ([\d]*)'
        pages_tag = thread_soup.find('td', text = re.compile(pages_regex))
        top_n_pages = int(re.findall(pages_regex, pages_tag.text)[0])
        print('found pages', top_n_pages)
    except:
        top_n_pages = 1
        print('Pagination wasn\'t foudn on the page. Load only one')

    return top_n_pages

In [11]:
def get_user_info_from_page(soup):
    posts = get_posts_from_page(soup)
    return [get_user_info_from_post(post) for post in posts]

In [52]:
def get_thread_msgs(thread_url, top_n_pages=5, save_path=None, close_page=True):
    thread_soup, driver = read_html_with_webdriver(thread_url, )
    
    thread_num = re.findall('\d{7}', thread_url)[0]
    print('Thread num:', thread_num)
    
    if any('Thread_'+thread_num in filename for filename in os.listdir(save_path)):
        matching_files = [f for f in os.listdir(save_path) if 'Thread_'+thread_num in f]
        print('The thread is already processed. Files:\n', matching_files)
        driver.close()
        print('Return the first one', matching_files[0])
        return pd.read_csv(os.path.join(save_path, matching_files[0]))
    
    result_df = pd.DataFrame()
    
    if top_n_pages == -1:
        print('top_n_pages = -1, so will scrap all pages')
        top_n_pages = get_pages_num(thread_soup)
        
#         if top_n_pages < 1000:
#             print('less than 1000 pages')
#             driver.close()
#             return None # mb return full file is exists
    
    if save_path is not None:
        save_frequency = get_save_freq(top_n_pages)
    
    for i in tqdm(range(1, top_n_pages+1), total=top_n_pages, mininterval=10):            
        all_info_from_page = get_all_info_from_page(thread_soup)
        result_df = pd.concat([result_df, all_info_from_page], ignore_index=True)        
        
        if i % save_frequency == 0 and save_path is not None:
            print("Saving after {}-th epoch".format(i))
            result_df.to_csv(os.path.join(save_path, 'Thread_'+thread_num+'_{}_pages.csv'.format(i)), index=False)

        # where is no page after the last one
        if i != top_n_pages:
            move_to_next_page(driver)
            thread_soup = driver2soup(driver)
    
    if close_page:
        driver.close()
        
    if save_path is not None:
        result_df.to_csv(os.path.join(save_path, 'Thread_'+thread_num+'_full_{}_pages.csv'.format(top_n_pages)), 
                         index=False)
        temp_files = [file for file in os.listdir(save_path) if thread_num in file and 'full' not in file]
        [os.remove(os.path.join(save_path, file)) for file in temp_files]
        
    return result_df

## Parsing one thread directly to csv 

In [14]:
thread_url = 'https://forum.littleone.ru/showthread.php?t=7773530'

In [35]:
%%time

result_df = get_thread_msgs(thread_url, top_n_pages=top_n_pages, 
                            save_path=data_path_csv)

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

Thread num: 7773530
top_n_pages = -1, so will scrap all pages
found pages 7


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.16s/it]

Unable to get next page
Wall time: 20.8 s


In [36]:
result_df.shape, result_df.memory_usage().sum()

((66, 13), 6530)

check the save query for just loaded file

In [37]:
result_df = get_thread_msgs(thread_url, top_n_pages=top_n_pages, 
                            save_path=data_path_csv)
result_df.shape, result_df.memory_usage().sum()

Thread num: 7773530
The thread is already processed. Files:
 ['Thread_7773530_full_7_pages.csv']
Return the first one Thread_7773530_full_7_pages.csv


((66, 13), 6530)

## Parsing a subsection (collection of threads) directly to csv

In [38]:
subsection_url = 'https://forum.littleone.ru/forumdisplay.php?f=322'

In [40]:
# TODO: a folder for the whole subsection may be created
def get_subsection_msgs(subsection_url, **kwargs):
    soup, driver = read_html_with_webdriver(subsection_url)
    topics = soup.find('ol', {'id': 'threads'}).find_all('li', {'class': re.compile('threadbit hot *')})
    for topic in topics:
        print("parsing topic: ", topic.find('a', 'title').text)
        href = topic.find('a', href=True).attrs['href']
        get_thread_msgs('https://forum.littleone.ru/'+href, **kwargs);
    driver.close()

In [47]:
get_subsection_msgs(subsection_url, **dict(top_n_pages=-1, save_path=data_path_csv))

parsing topic:  Про питание ребенка - очень важно!
Thread num: 8454147
The thread is already processed. Files:
 ['Thread_8454147_full_22_pages.csv']
Return the first one Thread_8454147_full_22_pages.csv
parsing topic:  Что производят в Петербурге для детей?
Thread num: 8218973
The thread is already processed. Files:
 ['Thread_8218973_full_15_pages.csv']
Return the first one Thread_8218973_full_15_pages.csv
parsing topic:  Как научить ребенка есть овощи и фрукты в 5 лет?
Thread num: 8424083
The thread is already processed. Files:
 ['Thread_8424083_full_6_pages.csv']
Return the first one Thread_8424083_full_6_pages.csv
parsing topic:  Детские рецепты
Thread num: 8211736
The thread is already processed. Files:
 ['Thread_8211736_full_16_pages.csv']
Return the first one Thread_8211736_full_16_pages.csv
parsing topic:  Питание аллергика в детском саду
Thread num: 8461409
The thread is already processed. Files:
 ['Thread_8461409_full_2_pages.csv']
Return the first one Thread_8461409_full_2_pa

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Thread num: 8414736
top_n_pages = -1, so will scrap all pages
Pagination wasn't foudn on the page. Load only one
parsing topic:  Молочные смеси



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Thread num: 8381500
top_n_pages = -1, so will scrap all pages
Pagination wasn't foudn on the page. Load only one
parsing topic:  Ввод продуктов после аллергии



  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Thread num: 8272742
top_n_pages = -1, so will scrap all pages
found pages 3


 33%|████████████████████████████                                                        | 1/3 [00:01<00:02,  1.29s/it]


AssertionError: Posts not found on page. Soup:
<html dir="ltr" id="vbulletin_html" lang="ru" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="https://forum.littleone.ru" id="e_vb_meta_bburl" name="vb_meta_bburl"/>
<base href="https://forum.littleone.ru/"/><!--[if IE]></base><![endif]-->
<meta content="vBulletin 4.2.1" name="generator"/>
<meta content="width=1200" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="322" name="forum_id"/>
<link href="favicon.ico" rel="Shortcut Icon" type="image/x-icon"/>
<script type="text/javascript">
<!--
	if (typeof YAHOO === 'undefined') // Load ALL YUI Local
	{
		document.write('<script type="text/javascript" src="clientscript/yui/yuiloader-dom-event/yuiloader-dom-event.js?v=422"><\/script>');
		document.write('<script type="text/javascript" src="clientscript/yui/connection/connection-min.js?v=422"><\/script>');
		var yuipath = 'clients

In [54]:
get_thread_msgs('https://forum.littleone.ru/showthread.php?t=8272742', top_n_pages=top_n_pages, 
                            save_path=data_path_csv)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Thread num: 8272742
top_n_pages = -1, so will scrap all pages
found pages 3


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.81it/s]


,post_text,user_title,user_name,user_link,Регистрация,Сообщений,guest,Адрес,post_datetime,answer_to_user,answer_to_post_head,answer_to_user_link,answer_to_post_link
0,\n\t\t\t\t\t\t\tВсем привет. Моя младшей малыш...,\n\t\t\t\tс Луны\n\t\t\t,-АпЕлЬсИнКа-,https://forum.littleone.ru/member.php?u=47470&...,24.09.2007,32183,False,NaN,2019-01-22 02:36:00,NaN,NaN,NaN,NaN
1,\n\t\t\t\t\t\t\tа врач то что говорит?\nобычно...,\n\t\t\t\tНаш человек\n\t\t\t,Мамулечка троих,https://forum.littleone.ru/member.php?u=842298...,02.04.2018,5041,False,NaN,2019-01-22 09:19:00,-АпЕлЬсИнКа-,Ввод продуктов после аллергии,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=16...
2,\n\n\n\n\n\n Сообщение от -АпЕлЬсИнКа-\n\n\nВс...,\n\t\t\t\tКосмонавт\n\t\t\t,Tatta1,https://forum.littleone.ru/member.php?u=501565...,15.02.2012,20873,False,Красногвардейский р-н,2019-01-22 10:08:00,-АпЕлЬсИнКа-,Ввод продуктов после аллергии,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=16...
3,\n\n\n\n\n\n Сообщение от Мамулечка троих\n\n\...,\n\t\t\t\tс Луны\n\t\t\t,-АпЕлЬсИнКа-,https://forum.littleone.ru/member.php?u=47470&...,24.09.2007,32183,False,NaN,2019-01-22 12:52:00,Tatta1,Приветик! \nЕсли кожа очистилась и в...,https://forum.littleone.ru/member.php?userid=5...,https://forum.littleone.ru/showthread.php?p=16...
4,\n\n\n\n\n\n Сообщение от -АпЕлЬсИнКа-\n\n\nВр...,\n\t\t\t\tКосмонавт\n\t\t\t,Tatta1,https://forum.littleone.ru/member.php?u=501565...,15.02.2012,20873,False,Красногвардейский р-н,2019-01-22 13:30:00,-АпЕлЬсИнКа-,Врачу неудобно писать и спрашивать... И...,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=16...
5,"\n\n\n\n\n\n Сообщение от Tatta1\n\n\nДа, если...",\n\t\t\t\tс Луны\n\t\t\t,-АпЕлЬсИнКа-,https://forum.littleone.ru/member.php?u=47470&...,24.09.2007,32183,False,NaN,2019-01-22 15:35:00,Tatta1,"Да, если была аллергия, то примерно 2...",https://forum.littleone.ru/member.php?userid=5...,https://forum.littleone.ru/showthread.php?p=16...
6,"\n\t\t\t\t\t\t\tпримерно так. главное, чтобы п...",\n\t\t\t\tНаш человек\n\t\t\t,Мамулечка троих,https://forum.littleone.ru/member.php?u=842298...,02.04.2018,5041,False,NaN,2019-01-22 15:46:00,-АпЕлЬсИнКа-,Врачу неудобно писать и спрашивать... И...,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=16...
7,\n\n\n\n\n\n Сообщение от Мамулечка троих\n\n\...,\n\t\t\t\tс Луны\n\t\t\t,-АпЕлЬсИнКа-,https://forum.littleone.ru/member.php?u=47470&...,24.09.2007,32183,False,NaN,2019-01-22 21:25:00,Мамулечка троих,"примерно так. главное, чтобы потом не...",https://forum.littleone.ru/member.php?userid=8...,https://forum.littleone.ru/showthread.php?p=16...
8,\n\t\t\t\t\t\t\tНа витамин Д часто бывает алле...,\n\t\t\t\tМега-элита\n\t\t\t,Наталинка,https://forum.littleone.ru/member.php?u=715117...,16.11.2014,4890,False,Приморский,2019-01-22 22:34:00,-АпЕлЬсИнКа-,Ввод продуктов после аллергии,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=16...
9,\n\n\n\n\n\n Сообщение от Наталинка\n\n\nНа ви...,\n\t\t\t\tс Луны\n\t\t\t,-АпЕлЬсИнКа-,https://forum.littleone.ru/member.php?u=47470&...,24.09.2007,32183,False,NaN,2019-01-23 00:15:00,Наталинка,"На витамин Д часто бывает аллергия,...",https://forum.littleone.ru/member.php?userid=7...,https://forum.littleone.ru/showthread.php?p=16...


In [ ]:
pd.read_csv(os.path.join(data_path, 'Thread_8429026_full.csv'))

# Load link and process after

## Get all links for a subsection

https://youtu.be/nR8WhdcRJwM?t=4009

In [22]:
def get_topic_pages_num_from_subsection(topic):
    last_page_tag = topic.find("a", title=lambda title: title and title.startswith("Перейти"))
    #print('last_page_tag', last_page_tag)
    if last_page_tag is not None: # this may occur if there is no "go to specific page in preview"
        print('enough pages')
        return int(re.findall('стр. ([\d]*)', last_page_tag.attrs['title'])[0])
    
    pagination = topic.find('dl', {'class': 'pagination'})
    if pagination is not None:       
        print('use pagination \(few pages\)')
        return int(pagination.find('dd').find_all('span')[-1].text)
    
    print('only one page')
    return 1

In [23]:
# rename to more specific
def get_link(forum, thread_num, page):
    return forum + 'showthread.php?t=' + str(thread_num) + '&page=' + str(page)

In [24]:
def download_subsection_threads(subsection_url, subsection_pages=2, **kwargs):
    soup, driver = read_html_with_webdriver(subsection_url, headless=True)
    for subsection_page in range(subsection_pages):
        print(f'\n\n{subsection_page + 1} page in subsection')
        topics = soup.find('ol', {'id': 'threads'}).find_all('li', {'class': re.compile('threadbit hot *')})
        for topic in topics:
            pages = get_topic_pages_num_from_subsection(topic)
            topic_name = topic.find('a', 'title').text
            href = topic.find('a', href=True).attrs['href']
            #print('href', href)
            topic_num = re.findall('\d{7}', href)[0]
            print("topic_num: ", topic_num, "topic name: ", topic_name, "pages:", pages)
            topic_local_dir = os.path.join(raw_data_path, str(topic_num))
            if not os.path.exists(topic_local_dir):
                os.makedirs(topic_local_dir)
            else:
                print('already exists')
                continue

            for i in tqdm_notebook(range(1, pages + 1), total=pages):
                wget.download(get_link('https://forum.littleone.ru/',topic_num, i),
                              out=os.path.join(topic_local_dir, str(i)))

        move_to_next_page(driver)
        soup = driver2soup(driver)
    driver.close()

In [39]:
subsection_url = 'https://forum.littleone.ru/forumdisplay.php?f=264'

In [41]:
download_subsection_threads(subsection_url=subsection_url, subsection_pages=10)



1 page in subsection
enough pages
topic_num:  8430795 topic name:  ДевятиКЛАССные будни 2020-2021:))) pages: 289
already exists
enough pages
topic_num:  8447234 topic name:  Коронный выпуск 2020, держим строй! pages: 226
already exists
enough pages
topic_num:  5422320 topic name:  куда поступают подростки у которых невысокие баллы по ЕГЭ pages: 1728
already exists
enough pages
topic_num:  6941702 topic name:  Поступили в ВШЭ...все сюда pages: 347
already exists
enough pages
topic_num:  8429026 topic name:  11 класс - Финишная прямая 2020-2021 pages: 100
already exists
enough pages
topic_num:  7376205 topic name:  ЛЭТИ 1курс.Кто с нами? pages: 529
already exists
enough pages
topic_num:  8447227 topic name:  Будни и праздники студентов СПО pages: 54
already exists
enough pages
topic_num:  8427171 topic name:  ^ↀᴥↀ^_10_КЛАСС_2020-2021_^ↀᴥↀ^ pages: 381
already exists
enough pages
topic_num:  8225593 topic name:  Питерские родители московских студентов. pages: 375
already exists
enough pa


use pagination \(few pages\)
topic_num:  8446363 topic name:  Информатика и физика, с чего начать изучение для поступления? pages: 6



enough pages
topic_num:  8343677 topic name:  Будущим абитуриентам колледжей. Проходные баллы. pages: 43



use pagination \(few pages\)
topic_num:  8139455 topic name:  Некрасовский педколледж pages: 3



enough pages
topic_num:  8205988 topic name:  Пусть и про БОНЧ будет тема :) pages: 36


...


use pagination \(few pages\)
topic_num:  8199839 topic name:  Гидромет (РГГМУ) pages: 4



use pagination \(few pages\)
topic_num:  8197314 topic name:  Ранхигс или СПбГЭУ (экс-Финэк/Инжекон)?  Помогите с принятием решения. pages: 2



use pagination \(few pages\)
topic_num:  6300374 topic name:  Прошу отзывов о Политехническом колледже городского хозяйства!! pages: 4



use pagination \(few pages\)
topic_num:  7291768 topic name:  Если летом уже 17 лет , посоветуйте лагерь pages: 6



use pagination \(few pages\)
topic_num:  8159699 topic name:  Нарвский Колледж Тартуского Университета/ бесплатное обучение pages: 3


## Processing loaded links

In [15]:
%%time

if not os.path.exists(data_path_csv):
    os.makedirs(data_path_csv)
else:
    print(data_path_csv, 'already exists')

def process_thread(thread_num, raw_data_path, data_path_csv):
    result_df = pd.DataFrame()
    print('thread_num', thread_num)

    if any('Thread_' + thread_num in s for s in os.listdir(data_path_csv)):
        print('already processed')
        return 0

    pages = os.listdir(os.path.join(raw_data_path, thread_num))
    for file in pages:
        all_info_from_page = get_all_info_from_file(os.path.join(raw_data_path, thread_num, file))
        result_df = pd.concat([result_df, all_info_from_page], ignore_index=True)

    result_df.to_csv(os.path.join(data_path_csv, 'Thread_' + thread_num + \
                                  '_full_{}_pages.csv'.format(len(pages))),
                     index=False)
    return 1
    
    
for thread_num in os.listdir(raw_data_path):
    process_thread(thread_num, raw_data_path, data_path_csv)

D:\Data\littleone_csv_with_names already exists
thread_num 3275945


D:\Programs\IDE_and_for_programming\Anaconda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



thread_num 4251847
thread_num 5408459
thread_num 5422320
thread_num 5534754
thread_num 5599851
thread_num 5600277
thread_num 6056812
thread_num 6080109
thread_num 6300374
thread_num 6439904
thread_num 6522944
thread_num 6796713
thread_num 6857993
thread_num 6868132
thread_num 6874132
thread_num 6876859
thread_num 6877836
thread_num 6892264
thread_num 6912352
thread_num 6924497
thread_num 6928444
thread_num 6941702
thread_num 7103863
thread_num 7144336
thread_num 7163483
thread_num 7204711
thread_num 7271618
thread_num 7291768
thread_num 7305262
thread_num 7309580
thread_num 7319331
thread_num 7324410
thread_num 7339300
thread_num 7376205
thread_num 7415337
thread_num 7515355
thread_num 7539634
thread_num 7609612
thread_num 7617457
thread_num 7641000
thread_num 7717404
thread_num 7718036
thread_num 7720107
thread_num 7741352
thread_num 7764006
thread_num 7782703
thread_num 7785682
thread_num 7790799
thread_num 7905320
thread_num 7923555
thread_num 7979477
thread_num 7983177
thread_num 7

In [14]:
%%time

from multiprocessing import Pool
import multiprocessing as mp
pool = mp.Pool(1)

threads = os.listdir(raw_data_path)
for thread_num in threads:
    pool.imap_unordered(process_thread, args=(thread_num, raw_data_path, data_path_csv))
pool.close()
pool.join()

TypeError: imap_unordered() got an unexpected keyword argument 'args'

In [104]:
get_all_info_from_file(os.path.join(raw_data_path, str(4251847), str(374)))

,post_text,user_title,user_name,user_link,Регистрация,Сообщений,guest,Адрес,post_time,answer_to_user,answer_to_post_head,answer_to_user_link,answer_to_post_link
0,\n\t\t\t\t\t\t\tСпасибо)))\n\t\t\t\t\t\t,\n\t\t\t\tГость\n\t\t\t,Olia1405,https://forum.littleone.ru/member.php?u=755127...,23.07.2015,5,False,NaN,2017-12-05 15:26:00,TatianaZ,Можно подавать в несколько...,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=15...
1,"\n\t\t\t\t\t\t\tЯ не ошибаюсь, в прошлом году ...",\n\t\t\t\tGuest\n\t\t\t,Верлиоко,NaN,NaN,NaN,True,NaN,2017-05-14 12:46:00,ЗНАЮ МЕРУ,Измайловский 10. Телефон: (812)...,https://forum.littleone.ru/member.php?userid=7...,https://forum.littleone.ru/showthread.php?p=15...
2,\n\n\n\n\n\n Сообщение от Еелена\n\n\nЗдравств...,\n\t\t\t\tЭлита\n\t\t\t,TatianaZ,https://forum.littleone.ru/member.php?u=426124...,16.06.2011,2404,False,NaN,2017-05-15 12:06:00,Olia1405,Спасибо))),https://forum.littleone.ru/member.php?userid=7...,https://forum.littleone.ru/showthread.php?p=15...
3,\n\n\n\n\n\n Сообщение от амстель\n\n\nЯ не ош...,\n\t\t\t\tЭлита\n\t\t\t,TatianaZ,https://forum.littleone.ru/member.php?u=426124...,16.06.2011,2404,False,NaN,2017-05-15 12:10:00,Верлиоко,"Я не ошибаюсь, в прошлом году на дод в...",NaN,https://forum.littleone.ru/showthread.php?p=15...
4,\n\n\n\n\n\n Сообщение от TatianaZ\n\n\nВ ГУАП...,\n\t\t\t\tBanned\n\t\t\t,##NV##,https://forum.littleone.ru/member.php?u=146690...,17.03.2009,35965,False,м.Московская,2017-05-15 12:21:00,TatianaZ,В ГУАПе...,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=15...
5,\n\t\t\t\t\t\t\tДобрый день!\nПо целевому для ...,\n\t\t\t\tВетеран\n\t\t\t,La Vanda,https://forum.littleone.ru/member.php?u=796506...,24.05.2016,849,False,NaN,2017-05-15 12:57:00,natalla,Поступление по целевому,https://forum.littleone.ru/member.php?userid=2...,https://forum.littleone.ru/showthread.php?p=62...
6,\n\n\n\n\n\n Сообщение от La Vanda\n\n\nДобрый...,\n\t\t\t\tЭлита\n\t\t\t,TatianaZ,https://forum.littleone.ru/member.php?u=426124...,16.06.2011,2404,False,NaN,2017-05-15 13:11:00,La Vanda,"Добрый день! \nПо целевому для ""Горного""...",https://forum.littleone.ru/member.php?userid=7...,https://forum.littleone.ru/showthread.php?p=15...
7,\n\n\n\n\n\n Сообщение от La Vanda\n\n\nДобрый...,\n\t\t\t\tЭлита\n\t\t\t,TatianaZ,https://forum.littleone.ru/member.php?u=426124...,16.06.2011,2404,False,NaN,2017-05-15 14:48:00,La Vanda,"Добрый день! \nПо целевому для ""Горного""...",https://forum.littleone.ru/member.php?userid=7...,https://forum.littleone.ru/showthread.php?p=15...
8,\n\n\n\n\n\n Сообщение от TatianaZ\n\n\nВ ГУАП...,\n\t\t\t\tBanned\n\t\t\t,##NV##,https://forum.littleone.ru/member.php?u=146690...,17.03.2009,35965,False,м.Московская,2017-05-15 14:57:00,TatianaZ,В ГУАПе...,https://forum.littleone.ru/member.php?userid=4...,https://forum.littleone.ru/showthread.php?p=15...
9,\n\n\n\n\n\n Сообщение от ##NV##\n\n\nскажите ...,\n\t\t\t\tЭлита\n\t\t\t,TatianaZ,https://forum.littleone.ru/member.php?u=426124...,16.06.2011,2404,False,NaN,2017-05-15 15:09:00,##NV##,"скажите пожалуйста,а на дне открытых...",https://forum.littleone.ru/member.php?userid=1...,https://forum.littleone.ru/showthread.php?p=15...
